In [24]:
import numpy as np
from flask import Flask,request,jsonify,render_template
import pickle 
from datetime import datetime,date

from fbprophet import Prophet

app=Flask(__name__)
model_confirm=pickle.load(open('model_confirmed.pkl','rb'))
model_death=pickle.load(open('model_death.pkl','rb'))
model_recovered=pickle.load(open('model_recovered.pkl','rb'))

@app.route('/')
def home():
  return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
  dates=request.form.values()
  lst = list(dates)
  dates=datetime.strptime(lst[-1],'%Y-%m-%d').date()
  delta1=dates-date(2020,4,22)
  periods1=delta1.days
  delta2=dates-date(2020,4,20)
  periods2=delta2.days
  #Confirm prediction
  future_confirm = model_confirm.make_future_dataframe(periods=periods1)
  forecast=model_confirm.predict(future_confirm)
  yhat_confirm=forecast['yhat'].iloc[-1]
  yhat_confirm=round(yhat_confirm)
  yhat_lower_confirm=forecast['yhat_lower'].iloc[-1]
  yhat_upper_confirm=forecast['yhat_upper'].iloc[-1]
  # Death Prediction
  future_death = model_death.make_future_dataframe(periods=periods2)
  forecast=model_death.predict(future_death)
  yhat_death=forecast['yhat'].iloc[-1]
  yhat_death=round(yhat_death)
  yhat_lower_death=forecast['yhat_lower'].iloc[-1]
  yhat_upper_death=forecast['yhat_upper'].iloc[-1]
  #Recovered prediction
  future_recovered = model_recovered.make_future_dataframe(periods=periods2)
  forecast=model_recovered.predict(future_recovered)
  yhat_recovered=forecast['yhat'].iloc[-1]
  yhat_recovered=round(yhat_recovered)
  yhat_lower_recovered=forecast['yhat_lower'].iloc[-1]
  yhat_upper_recovered=forecast['yhat_upper'].iloc[-1]
  #Active Prediction
  yhat_active=yhat_confirm -(yhat_death+yhat_recovered)

  output=[yhat_confirm,yhat_active,yhat_recovered,yhat_death]
  return render_template('index.html', prediction_text='predicition should be $ {}'.format(output))

if __name__=='__main__':
  app.run(debug=True)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: ignored